In [12]:
!pip install pymongo


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import pymongo

In [ ]:
#1. Write a Python program to connect to MySQL and print the list of all available databases.
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['school_db']
collection = client['school_db']
print("Database created........")
print("List of databases after creating new one")
print(client.list_database_names())

Database created........
List of databases after creating new one
['admin', 'config', 'local']


In [15]:
import pandas as pd

In [16]:
students=pd.read_csv("./students.csv")
courses=pd.read_csv("./courses.csv")
enrollment=pd.read_csv("./enrollments.csv")

In [ ]:
#2.Create a function in Python to create the tables (students, courses, enrollments) in school_db.
#3Write a Python program to insert multiple students and courses into the respective tables using executemany().
#for Insert multiple docs → insert_many().

students_data = students.to_dict(orient="records")
courses_data = courses.to_dict(orient="records")
enrollments_data = enrollment.to_dict(orient="records")

db.students.insert_many(students_data)
db.courses.insert_many(courses_data)
db.enrollments.insert_many(enrollments_data)

print("Data imported into school_db successfully!")

Data imported into school_db successfully!


In [19]:
#4. Fetch and print all rows from the students table in a tabular format.

# Fetch all documents from students collection
students = list(db.students.find({}, {"_id": 0}))  # exclude _id for cleaner table

# Convert to DataFrame for tabular display
df = pd.DataFrame(students)

print("Students Table:\n")
print(df.to_string(index=False))  # print without index

Students Table:

 student_id     name  age grade                email
          1    Zane1   18     C     zane1@school.edu
          2     Uma2   23     A     uma2@outlook.com
          3   Wendy3   22     B     wendy3@yahoo.com
          4  Xander4   22     A  xander4@outlook.com
          5  Victor5   19     A    victor5@gmail.com
          6     Ian6   19     D     ian6@outlook.com
          7     Leo7   22     A       leo7@gmail.com
          8    Paul8   24     B    paul8@outlook.com
          9   Wendy9   23     B    wendy9@school.edu
         10    Leo10   23     A      leo10@yahoo.com
         11    Uma11   18     C     uma11@school.edu
         12   Rita12   18     B   rita12@outlook.com
         13   Yara13   22     B     yara13@yahoo.com
         14    Bob14   19     C    bob14@outlook.com
         15 Quincy15   24     A  quincy15@school.edu
         16   Tina16   19     A     tina16@yahoo.com
         17  Alice17   25     B  alice17@outlook.com
         18    Sam18   20    

In [20]:
#5. Write a program to update the grade of a student (given student_id) via a Python input prompt.

student_id = input("Enter Student ID to update: ")
new_grade = input("Enter new grade: ")

# Update student's grade
result = db.students.update_one(
    {"student_id": student_id},   # filter
    {"$set": {"grade": new_grade}}  # update
)

# Feedback
if result.matched_count > 0:
    print("Grade updated successfully!")
else:
    print("Student not found.")

Student not found.


In [26]:
#6 Delete a student from the students table using Python input for student_id.

student_id = input("Enter Student ID to delete: ")

# Delete student
result = db.students.delete_one({"student_id": student_id})

# Feedback
if result.deleted_count > 0:
    print("Student deleted successfully!")
else:
    print("Student not found.")

Student not found.


In [31]:
#7  Write a Python script to insert enrollment records linking students with their courses, ensuring foreign key integrity.
#just insert into an enrollments collection (foreign keys not enforced, but can be validated in code).

student_id = int(input("Enter student ID: "))
course_id = int(input("Enter course ID: "))

# Check existence
student = db.students.find_one({"student_id": student_id})
course = db.courses.find_one({"course_id": course_id})

if student and course:
    db.enrollments.insert_one({
        "student_id": student_id,
        "course_id": course_id
    })
    print("Enrollment added successfully!")
else:
    print("Student or Course not found!")

Enrollment added successfully!


In [33]:
#8. Fetch all students enrolled in a particular course (given course_id as input).

course_id = int(input("Enter course ID: "))

# Step 1: Find all enrollments for the course
enrollments = db.enrollments.find({"course_id": course_id}, {"student_id": 1, "_id": 0})

# Step 2: Fetch matching students
student_ids = [e["student_id"] for e in enrollments]

if student_ids:
    students = db.students.find({"student_id": {"$in": student_ids}}, {"_id": 0})
    print("\nStudents enrolled in course", course_id)
    for s in students:
        print(s)
else:
    print("No students enrolled in this course.")


Students enrolled in course 10
{'student_id': 1, 'name': 'Zane1', 'age': 18, 'grade': 'C', 'email': 'zane1@school.edu'}
{'student_id': 4, 'name': 'Xander4', 'age': 22, 'grade': 'A', 'email': 'xander4@outlook.com'}
{'student_id': 5, 'name': 'Victor5', 'age': 19, 'grade': 'A', 'email': 'victor5@gmail.com'}
{'student_id': 12, 'name': 'Rita12', 'age': 18, 'grade': 'B', 'email': 'rita12@outlook.com'}
{'student_id': 13, 'name': 'Yara13', 'age': 22, 'grade': 'B', 'email': 'yara13@yahoo.com'}
{'student_id': 20, 'name': 'Victor20', 'age': 19, 'grade': 'D', 'email': 'victor20@gmail.com'}
{'student_id': 22, 'name': 'Victor22', 'age': 18, 'grade': 'A', 'email': 'victor22@yahoo.com'}
{'student_id': 41, 'name': 'Ian41', 'age': 24, 'grade': 'D', 'email': 'ian41@outlook.com'}
{'student_id': 42, 'name': 'Quincy42', 'age': 24, 'grade': 'C', 'email': 'quincy42@gmail.com'}
{'student_id': 45, 'name': 'Xander45', 'age': 23, 'grade': 'C', 'email': 'xander45@yahoo.com'}
{'student_id': 46, 'name': 'Zane46', '

In [41]:
# checking
print("Sample students IDs:")
for s in db.students.find({}, {"student_id": 1, "_id": 0}).limit(5):
    print(s)

print("\nSample enrollment student IDs:")
for e in db.enrollments.find({}, {"student_id": 1, "_id": 0}).limit(5):
    print(e)


Sample students IDs:
{'student_id': '1'}
{'student_id': '2'}
{'student_id': '3'}
{'student_id': '4'}
{'student_id': '5'}

Sample enrollment student IDs:
{'student_id': '1'}
{'student_id': '2'}
{'student_id': '3'}
{'student_id': '4'}
{'student_id': '5'}


In [45]:
#9. Write a program to display students who are not enrolled in any course


#Step 1: Get all student_ids who are enrolled
enrolled_ids = [e["student_id"] for e in db.enrollments.find({}, {"student_id": 1, "_id": 0})]

# Step 2: Find students not in that list
students_no_courses = db.students.find({"student_id": {"$nin": enrolled_ids}}, {"_id": 0})

print("Students not enrolled in any course:\n")
for student in students_no_courses:
    print(student)

Students not enrolled in any course:



In [46]:
#10. Perform an INNER JOIN between students and courses using Python to show student names along with the courses they are enrolled in.

pipeline = [
    # Join students with enrollments
    {"$lookup": {
        "from": "enrollments",
        "localField": "student_id",
        "foreignField": "student_id",
        "as": "enrollments"
    }},
    {"$unwind": "$enrollments"},  # only keep students with enrollments (INNER JOIN)
    
    # Join with courses
    {"$lookup": {
        "from": "courses",
        "localField": "enrollments.course_id",
        "foreignField": "course_id",
        "as": "course_info"
    }},
    {"$unwind": "$course_info"},  # one student-course per doc

    # Select only relevant fields
    {"$project": {
        "_id": 0,
        "Student Name": "$name",
        "Course Name": "$course_info.course_name"
    }}
]

results = db.students.aggregate(pipeline)

# Print results
print("Student Name -> Course Name")
for r in results:
    print(f"{r['Student Name']} -> {r['Course Name']}")

Student Name -> Course Name
Zane1 -> Statistics
Zane1 -> Statistics
Uma2 -> Economics
Wendy3 -> English Literature
Xander4 -> Statistics
Victor5 -> Statistics
Victor5 -> Chemistry
Victor5 -> Economics
Ian6 -> Computer Science
Ian6 -> Biology
Ian6 -> History
Leo7 -> Mathematics
Paul8 -> Philosophy
Wendy9 -> Philosophy
Leo10 -> Computer Science
Leo10 -> Chemistry
Leo10 -> Mathematics
Uma11 -> Chemistry
Uma11 -> Economics
Uma11 -> English Literature
Rita12 -> Statistics
Yara13 -> Physics
Yara13 -> Philosophy
Yara13 -> Statistics
Bob14 -> Chemistry
Bob14 -> Mathematics
Quincy15 -> English Literature
Quincy15 -> History
Quincy15 -> Physics
Tina16 -> Economics
Tina16 -> Biology
Tina16 -> Mathematics
Alice17 -> Economics
Alice17 -> Mathematics
Alice17 -> Computer Science
Sam18 -> Chemistry
Paul19 -> Philosophy
Paul19 -> Mathematics
Victor20 -> English Literature
Victor20 -> Biology
Victor20 -> Statistics
Frank21 -> Computer Science
Frank21 -> Economics
Victor22 -> Chemistry
Victor22 -> Statis

In [48]:
#11. Write a Python function to search for a student by name (partial matches allowed using LIKE).

partial_name = input("Enter student name (partial search): ")

# query with regex (case-insensitive)
query = {"name": {"$regex": partial_name, "$options": "i"}}
students = db.students.find(query, {"_id": 0})

# check results
found = False
print("\nSearch results:\n")
for student in students:
    found = True
    print("ID:", student.get("student_id"),
          "| Name:", student.get("name"),
          "| Age:", student.get("age"),
          "| Grade:", student.get("grade"),
          "| Email:", student.get("email"))

if not found:
    print("No matching student found.")


Search results:

ID: 11 | Name: Uma11 | Age: 18 | Grade: C | Email: uma11@school.edu


In [50]:
#12. Add a new column email to the students table using Python and update it with random values.
import numpy as np

domains = np.array(["gmail.com", "yahoo.com", "outlook.com", "school.edu"])

# fetch all students
students = list(db.students.find())

for student in students:
    # random 3-digit number
    rand_num = np.random.randint(100, 999)
    # random domain
    domain = np.random.choice(domains)
    # base part from name
    name_part = student["name"].lower().replace(" ", ".")
    # construct email
    email = f"{name_part}{rand_num}@{domain}"

    # update document
    db.students.update_one(
        {"_id": student["_id"]},
        {"$set": {"email": email}}
    )

print("Emails added successfully")

Emails added successfully


In [51]:
#13. Create a program to count how many students are enrolled per course and display results in descending order.

# aggregation to count students per course
pipeline = [
    {"$group": {"_id": "$course_id", "student_count": {"$sum": 1}}},
    {"$sort": {"student_count": -1}}
]

results = db.enrollments.aggregate(pipeline)

print("Students enrolled per course:\n")
for r in results:
    print("Course ID:", r["_id"], "| Students:", r["student_count"])


Students enrolled per course:

Course ID: 3 | Students: 28
Course ID: 9 | Students: 22
Course ID: 5 | Students: 21
Course ID: 7 | Students: 21
Course ID: 1 | Students: 21
Course ID: 10 | Students: 20
Course ID: 6 | Students: 20
Course ID: 4 | Students: 18
Course ID: 2 | Students: 16
Course ID: 8 | Students: 13


In [52]:
#14. Write a Python script that accepts a course name as input and returns all enrolled students with their grades.

course_name = input("Enter course name (partial allowed): ")

# find matching courses
for course in db.courses.find({"course_name": {"$regex": course_name, "$options": "i"}}):
    print(f"\nStudents enrolled in {course['course_name']}:\n")
    enrollments = db.enrollments.find({"course_id": course["course_id"]})
    found = False
    for e in enrollments:
        student = db.students.find_one({"student_id": e["student_id"]})
        if student:
            found = True
            print("ID:", student["student_id"],
                  "| Name:", student["name"],
                  "| Grade:", student["grade"])
    if not found:
        print("No students enrolled.")


Students enrolled in Physics:

ID: 13 | Name: Yara13 | Grade: B
ID: 15 | Name: Quincy15 | Grade: A
ID: 37 | Name: Ian37 | Grade: D
ID: 43 | Name: Quincy43 | Grade: D
ID: 47 | Name: Eva47 | Grade: D
ID: 52 | Name: Hannah52 | Grade: A
ID: 53 | Name: Bob53 | Grade: C
ID: 57 | Name: Nina57 | Grade: B
ID: 60 | Name: Yara60 | Grade: B
ID: 75 | Name: Quincy75 | Grade: D
ID: 78 | Name: Ian78 | Grade: D
ID: 83 | Name: Leo83 | Grade: C
ID: 90 | Name: Zane90 | Grade: D
ID: 91 | Name: Tina91 | Grade: D
ID: 96 | Name: Ian96 | Grade: C
ID: 99 | Name: Bob99 | Grade: C

Students enrolled in Philosophy:

ID: 8 | Name: Paul8 | Grade: B
ID: 9 | Name: Wendy9 | Grade: B
ID: 13 | Name: Yara13 | Grade: B
ID: 19 | Name: Paul19 | Grade: A
ID: 22 | Name: Victor22 | Grade: A
ID: 25 | Name: Quincy25 | Grade: A
ID: 26 | Name: Mona26 | Grade: D
ID: 31 | Name: Nina31 | Grade: B
ID: 32 | Name: Hannah32 | Grade: A
ID: 44 | Name: Tina44 | Grade: C
ID: 54 | Name: Leo54 | Grade: C
ID: 55 | Name: Yara55 | Grade: A
ID: 59

In [54]:
#15.Export the contents of the students table into a CSV file using Python.

# fetch all students
students = list(db.students.find({}, {"_id": 0}))

# convert to DataFrame
df = pd.DataFrame(students)

# export to CSV
df.to_csv("students.csv", index=False)

print("Exported students to students.csv")

Exported students to students.csv


In [55]:
#16. Write a Python program to import student data from a CSV file into the students table.

# read CSV into DataFrame
df = pd.read_csv("students.csv")

# convert DataFrame rows to dictionary
records = df.to_dict(orient="records")

# insert into MongoDB
db.students.insert_many(records)

print("Student data imported successfully from CSV!")

Student data imported successfully from CSV!


In [56]:
#17

#NULL CANT DO

In [ ]:
#18.Handle exceptions in Python to gracefully catch duplicate entries (e.g., inserting a student with the same ID)

from pymongo import MongoClient, errors




# insert with exception handling
for rec in records:
    try:
        db.students.insert_one(rec)
        print(f"Inserted: {rec['student_id']} - {rec['name']}")
    except errors.DuplicateKeyError:
        print(f"Duplicate found! Student ID {rec['student_id']} already exists.")

Duplicate found! Student ID 1 already exists.
Duplicate found! Student ID 2 already exists.
Duplicate found! Student ID 3 already exists.
Duplicate found! Student ID 4 already exists.
Duplicate found! Student ID 5 already exists.
Duplicate found! Student ID 6 already exists.
Duplicate found! Student ID 7 already exists.
Duplicate found! Student ID 8 already exists.
Duplicate found! Student ID 9 already exists.
Duplicate found! Student ID 10 already exists.
Duplicate found! Student ID 11 already exists.
Duplicate found! Student ID 12 already exists.
Duplicate found! Student ID 13 already exists.
Duplicate found! Student ID 14 already exists.
Duplicate found! Student ID 15 already exists.
Duplicate found! Student ID 16 already exists.
Duplicate found! Student ID 17 already exists.
Duplicate found! Student ID 18 already exists.
Duplicate found! Student ID 19 already exists.
Duplicate found! Student ID 20 already exists.
Duplicate found! Student ID 21 already exists.
Duplicate found! Stude

In [59]:
#19. Write a program to implement transactions in Python:
# Insert a student
# Enroll them in a course
# Roll back if the second query fails

with client.start_session() as session:
    try:
        # start transaction
        with session.start_transaction():
            # 1. Insert student
            student = {"student_id": "10", "name": "David", "age": 22, "grade": "B"}
            db.students.insert_one(student, session=session)

            # 2. Enroll them in a course
            # deliberately use wrong course_id to test rollback
            enrollment = {"enroll_id": "5001", "student_id": "10", "course_id": "C999"}
            db.enrollments.insert_one(enrollment, session=session)

            print("Transaction successful! Student and enrollment inserted.")

    except errors.PyMongoError as e:
        print("Transaction aborted due to error:", e)

Transaction aborted due to error: Transaction numbers are only allowed on a replica set member or mongos, full error: {'ok': 0.0, 'errmsg': 'Transaction numbers are only allowed on a replica set member or mongos', 'code': 20, 'codeName': 'IllegalOperation'}


In [60]:
#20. Close the database connection safely in Python, ensuring both cursor and connection are properly closed.

client.close()

print("MongoDB connection closed safely.")

MongoDB connection closed safely.
